# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
! pip install cartopy
! pip install geoviews
! pip install pyproj

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,293.94,53,75,3.60,CA,1722886558
1,1,margate,51.3813,1.3862,292.42,74,1,4.62,GB,1722886559
2,2,ushuaia,-54.8000,-68.3000,272.96,64,20,0.00,AR,1722886432
3,3,karwar,14.8000,74.1333,299.13,86,64,2.81,IN,1722886562
4,4,bethel,41.3712,-73.4140,305.40,57,0,3.60,US,1722886563


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# %%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country')

# Display the map
all_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']>27]

city_temp = city_temp0.loc[city_temp0['Max Temp']>21]

city_wind = city_temp.loc[city_temp['Wind Speed']>4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()
# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,north bend,43.4065,-124.2243,295.36,72,0,7.72,US,1722886610
89,89,vadso,70.0744,29.7487,285.64,87,0,5.14,NO,1722886661
141,141,pimentel,-6.8367,-79.9342,295.31,64,0,8.75,PE,1722886167
153,153,pirallahi,40.4723,50.3330,300.23,75,0,4.66,AZ,1722886734
161,161,nantucket,41.2835,-70.0995,302.18,75,0,5.14,US,1722886576
168,168,huarmey,-10.0681,-78.1522,293.43,71,0,6.11,PE,1722886751
175,175,guanica,17.9716,-66.9080,306.76,69,0,8.16,PR,1722886759
183,183,arraial do cabo,-22.9661,-42.0278,300.13,69,0,8.75,BR,1722886768
226,226,minas de marcona,-15.2119,-75.1103,288.23,62,0,7.63,PE,1722886821
244,244,remire-montjoly,4.9167,-52.2667,305.17,66,0,6.17,GF,1722886842


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,north bend,US,43.4065,-124.2243,72,
89,vadso,NO,70.0744,29.7487,87,
141,pimentel,PE,-6.8367,-79.9342,64,
153,pirallahi,AZ,40.4723,50.3330,75,
161,nantucket,US,41.2835,-70.0995,75,
168,huarmey,PE,-10.0681,-78.1522,71,
175,guanica,PR,17.9716,-66.9080,69,
183,arraial do cabo,BR,-22.9661,-42.0278,69,
226,minas de marcona,PE,-15.2119,-75.1103,62,
244,remire-montjoly,GF,4.9167,-52.2667,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
north bend - nearest hotel: The Mill Casino Hotel
vadso - nearest hotel: Scandic Vadsø
pimentel - nearest hotel: Hospedaje Calle del Sol
pirallahi - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
huarmey - nearest hotel: Hostal Santa Rosa
guanica - nearest hotel: Copamarina Beach Resort
arraial do cabo - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
fort mcmurray - nearest hotel: Nomad Hotel and Suites
whitehorse - nearest hotel: Town & Mountain Hotel
geraldton - nearest hotel: Ocean Centre Hotel
morondava - nearest hotel: Hotel Sharon Menabe
truckee - nearest hotel: The Truckee Hotel
selebi-phikwe - nearest hotel: Cresta Bosele Hotel
hun - nearest hotel: فندق الهروج
sao joao da barra - nearest hotel: Pousada Mediterrâneo
lompoc - nearest hotel: Red Roof Inn Lompoc
pisco - nearest hotel: La Portada
armacao de buzios - nearest hotel: Vila da Santa
chivhu - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
45,north bend,US,43.4065,-124.2243,72,The Mill Casino Hotel
89,vadso,NO,70.0744,29.7487,87,Scandic Vadsø
141,pimentel,PE,-6.8367,-79.9342,64,Hospedaje Calle del Sol
153,pirallahi,AZ,40.4723,50.3330,75,No hotel found
161,nantucket,US,41.2835,-70.0995,75,No hotel found
168,huarmey,PE,-10.0681,-78.1522,71,Hostal Santa Rosa
175,guanica,PR,17.9716,-66.9080,69,Copamarina Beach Resort
183,arraial do cabo,BR,-22.9661,-42.0278,69,No hotel found
226,minas de marcona,PE,-15.2119,-75.1103,62,No hotel found
244,remire-montjoly,GF,4.9167,-52.2667,66,Complexe Belova


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)